In [ ]:
import osmnx as ox
import networkx as nx
import heapq

In [ ]:
place_name = "nellore, India" 

In [ ]:
graph = ox.graph_from_place(place_name, network_type='drive')

In [ ]:
adj_matrix = nx.to_numpy_array(graph, weight='length')
V = adj_matrix.shape[0]

In [ ]:
def dijkstra(graph, V, source_index):
    value = [float('inf')] * V
    parent = [-1] * V
    processed = [False] * V

    value[source_index] = 0
    priority_queue = [(0, source_index)]

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if processed[current_node]:
            continue

        processed[current_node] = True

        for neighbor, weight in enumerate(adj_matrix[current_node]):
            if weight > 0 and not processed[neighbor]:
                new_distance = current_distance + weight

                if new_distance < value[neighbor]:
                    value[neighbor] = new_distance
                    parent[neighbor] = current_node
                    heapq.heappush(priority_queue, (new_distance, neighbor))

    return parent, value


In [ ]:
# GIVE YOUR DESIRED CO_ORDINATES
source_coords = (14.396820234732408, 79.98401219331771)  
destination_coords = (14.465602873499273, 79.89001035729736)

source_node = ox.distance.nearest_nodes(graph, source_coords[1], source_coords[0])
destination_node = ox.distance.nearest_nodes(graph, destination_coords[1], destination_coords[0])

source_index = list(graph.nodes()).index(source_node)
destination_index = list(graph.nodes()).index(destination_node)

parent, value = dijkstra(adj_matrix, V, source_index)

In [ ]:
shortest_path = []
current = destination_index

while current != source_index:
    shortest_path.append(current)
    current = parent[current]

shortest_path.append(source_index)
shortest_path.reverse() 

total_distance = sum(adj_matrix[u, v] for u, v in zip(shortest_path[:-1], shortest_path[1:]))
print(f"Shortest path distance: {total_distance:.2f} meters")

shortest_path_node_ids = [list(graph.nodes())[index] for index in shortest_path]

fig, ax = ox.plot_graph_route(graph, shortest_path_node_ids, route_color='red', route_linewidth=3)

In [ ]:
import folium.plugins


map_center = [(source_coords[0] + destination_coords[0]) / 2, (source_coords[1] + destination_coords[1]) / 2]
m = folium.Map(location=map_center, zoom_start=14, tiles='cartodb positron')  

# Add the street network to the map
for u, v, key, data in graph.edges(data=True, keys=True):
    if 'geometry' in data:
        coordinates = [(point[1], point[0]) for point in data['geometry'].coords]
        folium.PolyLine(coordinates, color='gray', weight=2, opacity=0.5).add_to(m)

# Converting the shortest path from node IDs to coordinates
shortest_path_coords = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in shortest_path_node_ids]

# Adding the shortest path to the map
folium.plugins.AntPath(shortest_path_coords, color='red', weight=3, opacity=0.7).add_to(m)

#markers for the source and destination nodes
folium.Marker(source_coords, icon=folium.Icon(color='green', icon='info-sign')).add_to(m)
folium.Marker(destination_coords, icon=folium.Icon(color='blue', icon='flag')).add_to(m)


#full screen
folium.plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)

print(f"Shortest path distance: {total_distance:.2f} meters")

m